# Topological Data Analysis
Matias Ollu, Yuguang Yao

## Abstact
This resipotary aims to present our result to solve the problem of the project of topological data analysis in CSC_42021_EP - Conception et analyse d'algorithmes (2024-2025) of the École Polytechnique. We present an alogritme to compute the simplexes and the filtration value of a given $\alpha$-complex or Čech-complex of a arbitary point-set. We devolepped an alogritme general enough to calculate the circumcircle of $n$ points in $d$-dimension euclidean space.

## Environment
We utilise the packets following:

In [1]:
import random
import numpy as np
from itertools import combinations

## $\alpha$-complex and Čech-complex

(À completer, les définitions strict et les theoremes utilisées, réference: https://courses.cs.duke.edu/fall06/cps296.1/)

We will firstly define the key concepts and give the theorem that we are going to use. And we will forlulize our problems formly.

* Define 1: 

## Question 1

>Task 1. Implement the computation of the minimal enclosing ball (MEB)
>of a set of points using one (or more) of the algorithms from https:// en.
>wikipedia.org/ wiki/ LP-type_problem.

We defined a class of Sphere(center,radius), and uitilised the alogithm of Weltz(LP-Problem) to solve the MEB quetion and we solved alse the circumcircle of $n$ points in $d$-dimension euclidean space.

In [2]:
# This class is used to represent a sphere.
class Sphere:
    def __init__(self, center, radius):
        self.center = np.array(center)
        self.radius = radius

    def contains(self, point):
        """Check if the sphere contains the given point."""
        return np.linalg.norm(self.center - np.array(point)) <= self.radius #+ 1e-9
    
    def contains_strict(self,point):
        """Check if the sphere contains the given point."""
        norme = np.linalg.norm(self.center - np.array(point))
        return not (np.isclose((norme),self.radius) or norme>self.radius+1e-9)
    
    def onradius(self,point):
        """Check if the point is on the sphere"""
        return np.isclose(np.linalg.norm(self.center - np.array(point)),self.radius)

On déduit les équation pour calculer des circumcircles:

Soit $P_i^n$ le $i$-ième coordinée de point $P^n$ dans un emsemble des $N$-points dans $d$-dimension en satisfasant $1 \leq n \leq N \leq d+1$. Et soit $c=\sum_n k_n P^{n0}$. Pour la centre du circumcircle, pour quelconque $n,m\in[1,n]\cap Z$  ,on a:

$$|c-P^n|=|c-P^m| $$
$$\sum_i (c_i-P_i^n)^2-(n\to m)=0 $$
$$\sum_i (P_i^n)^2+2P_i^nc_i-(n\to m)=0 $$
$$\sum_i ((P_i^n)^2-(P_i^m)^2)=\sum_l (P_i^{l0}\sum_i 2(P_i^n-P_i^m)) k_l  $$
$$|P^n|-|P^m|=\sum_l (P_i^{l0}\sum_i 2(P_i^n-P_i^m)) k_l  .$$

Dans l'implementation, on prends $m=0$
$$|P^n|-|P^0|=\sum_l (2P_i^l\sum_i (P_i^n-P_i^0)) k_l  $$

$$A_{mn}=2P^{m0} \cdot P^{n0} $$
$$b_n=|P^n|-|P^0|.$$

c'est une systeme linaire avec $n-1$ équations et $n-1$ variables(theorem à prouver). Apres qu'on resoudre la systeme, on peut extrait  $c=\sum_n k_n P^{n0}$ par $c=k \cdot P+P^0$.

In [3]:
def make_sphere_n_points(points):
    """
    Trouver le minimal circumcircle dans l'espace d-dimension de n points (n <= d+1).
    
    when utilising this mathod, we assume that the points are not colinear and that the dimension is at least 2.

    Paramètres :
        points: np.ndarray, shape (n, d)
    Retourne :
        Sphere
    """
    points = np.array(points, dtype=float)
    
    # Calcul de la matrice A et du vecteur b
    diffs = points[1:] - points[0]  # Différences (P^n - P^0) pour n = 1, ..., N-1
    A = 2 * np.dot(diffs, diffs.T)  # Matrice A
    b = np.sum(diffs ** 2, axis=1)  # Vecteur b

    # Résolution du système linéaire pour trouver les coefficients k
    k = np.linalg.solve(A, b)

    # Calculer le centre
    center = points[0] + np.dot(k, diffs)

    # Calculer le rayon
    radius = np.linalg.norm(center - points[0])

    # Retourner une instance de Sphere
    return Sphere(center=center, radius=radius)

In [4]:
def trivial(R):
    """Find the minimal sphere for 0, 1 or mores points."""
    if not R:
        return Sphere([0, 0, 0], 0)
    elif len(R) == 1:
        return Sphere(R[0], 0)
    elif len(R) >= 1:
        return make_sphere_n_points(R)

def welzl(P, R):
    """Recursive implementation of Welzl's algorithm for 3D."""
    if not P or len(R) == len(P[0])+   1 :
        return trivial(R)

    p = P.pop(random.randint(0, len(P) - 1))
    D = welzl(P, R)

    if D.contains(p):
        P.append(p)
        return D

    result = welzl(P, R + [p])
    P.append(p)
    return result

def minimal_enclosing_sphere(points):
    """Compute the minimal enclosing sphere for a set of points."""
    points = points[:]
    random.shuffle(points)
    return welzl(points, [])

In [5]:
def test_task1():
    """Test cases for minimal enclosing sphere."""
    # Test 1: Single point
    points = [(0, 0, 0)]
    sphere = minimal_enclosing_sphere(points)
    assert np.allclose(sphere.center, [0, 0, 0])
    assert np.isclose(sphere.radius, 0)
    print("Test 1.1 passed!")

    # Test 2: Two points
    points = [(0, 0, 0), (2, 0, 0)]
    sphere = minimal_enclosing_sphere(points)
    assert np.allclose(sphere.center, [1, 0, 0])
    assert np.isclose(sphere.radius, 1)
    print("Test 1.2 passed!")

    # Test 3: Three points
    points = [(-10, 0, 0), (10, 0, 0), (0, 1, 0)]
    sphere = minimal_enclosing_sphere(points)
    assert np.allclose(sphere.center, [0, 0, 0])
    assert np.isclose(sphere.radius, 10)
    print("Test 1.3 passed!")

    # Test 4: Four points
    points = [(5, 0, 1), (-1, -3, 4), (-1, -4, -3), (-1, 4, -3)]
    sphere =  minimal_enclosing_sphere(points)
    assert np.allclose(sphere.center, [0, 0, 0])
    assert np.isclose(sphere.radius, np.sqrt(26))
    print("Test 1.4 passed!")

    print("All test cases passed!")

test_task1()

Test 1.1 passed!
Test 1.2 passed!
Test 1.3 passed!
Test 1.4 passed!
All test cases passed!


In [6]:

def task2(points,emu):
    """Compute the filtration value for the points in emu"""
    points_chosen = [points[i] for i in emu]
    filtration = minimal_enclosing_sphere(points_chosen).radius

    return filtration

def enum_simplex2(points):
    "énumère et affiche les simplexes avec la valeur de filtrage"
    parties= [] #on fait la liste des sous ensembles de points:

    i, imax = 0, 2**len(points)-1 #on construit un itérateur i, dont les bits 1 seront les points sélectionnés dans le sous ensemble
    while i <= imax:
        s = []
        j, jmax = 0, len(points)-1
        while j <= jmax:
            if (i>>j)&1 == 1:
                s.append(points[j])
            j += 1
        parties.append(s)
        i += 1 

    #on affiche les simplexes avec filtration value

    for enum in parties:
        filtration = task2(points,enum)
        print(f"({enum}) -> {filtration}")

In [7]:
def test_task2():
    P = [(5, 0, 1), (-1, -3, 4), (-1, -4, -3), (-1, 4, -3)]

    test_cases = [
        ([0], 0),
        ([1], 0),
        ([2], 0),
        ([3], 0),
        ([2, 1], 3.53553),
        ([1, 0], 3.67425),
        ([3, 2], 4),
        ([2, 0], 4.12311),
        ([3, 0], 4.12311),
        ([2, 1, 0], 4.39525),
        ([3, 2, 0], 4.71495),
        ([3, 1], 4.94975),
        ([3, 2, 1], 5),
        ([3, 1, 0], 5.04975),
        ([3, 2, 1, 0], 5.09902),
    ]

    for enu, expected in test_cases:
        assert np.allclose(task2(P, enu), expected), f"Test({enu}) failed!"
        print(f"Test({enu}) passed!")


test_task2()

Test([0]) passed!
Test([1]) passed!
Test([2]) passed!
Test([3]) passed!
Test([2, 1]) passed!
Test([1, 0]) passed!
Test([3, 2]) passed!
Test([2, 0]) passed!
Test([3, 0]) passed!
Test([2, 1, 0]) passed!
Test([3, 2, 0]) passed!
Test([3, 1]) passed!
Test([3, 2, 1]) passed!
Test([3, 1, 0]) passed!
Test([3, 2, 1, 0]) passed!


In [8]:
def enum3(points):
    """
    Génère un tableau où chaque ligne correspond aux sous-ensembles d'une certaine taille.
    """
    n = len(points)
    return [[list(comb) for comb in combinations(range(n), k)] for k in range(1, n + 1)]
  
def task3_mathias(points,l):
    """implement an algorithm that enumerates the simplexes and their filtration values."""

    emu = enum3(points)
    simplex = {tuple([i]): 0 for i in range(len(points))} #on initialise le premier simplexe
    #IsSimplex = {0:True} #dictionnaire indiquant si un sous ensemble forme un simplexe
    IsSimplex= [[0] * len(sublist) for sublist in emu] # le tableau de suivi: 0 si jamais vu, 1 si simplexe, 2 si pas un simplexe

    n=len(points)


    #Vestion 1 Mathias: pas optimale, on évite pas tj de calculer les simplexes qui en contiennent d'autres
    for i in range(1,len(emu)):
        for j in range(len(emu[i])):
            test = IsSimplex[i][j]
            if(test==0): #pas encore connu
                filtration = task2(points,emu[i][j])
                if (filtration > l): #pas un simplexe
                    IsSimplex[i][j] = 2
                    if(i<n-1): #on s'assure qu'on est pas à la dernière ligne
                        for k in range(n-i-1-j):
                         IsSimplex[i+1][j+k]=2 #les sous ensembles de taille supérieure qui contiennent emu[i,j] ne sont pas non plus des simplexes
                else:
                    IsSimplex[i][j] = 1
                    simplex[tuple(emu[i][j])]=filtration
            elif(test==1): #est un simplexe -> normalement impossible de revenir sur nos pas !
                raise RecursionError("l'ago revient sur ses pas !")
            #Sinon, test ==2 et ce n'est pas un simplexe. on passe au prochain. Pas besoin de tester cette éventualité
    
    for key, value in simplex.items():
        print(f"{key} -> {value}")
    
    return simplex   

def task3(points,l):
    enum = enum3(points)
    IsSimplex = {tuple([i]): 1 for i in range(len(points))}

    simplex = {tuple([i]): Sphere(points[i], 0) for i in range(len(points))} #on initialise le premier simplexe

    
    for i in range(1,len(enum)):
        for j in range(len(enum[i-1])):
            current_simplex = enum[i-1][j]

            for k in range(len(points)):

                pn =tuple(current_simplex + [k])

                if k in current_simplex:
                    IsSimplex[pn] = 0
                    break

                if pn in simplex:
                    break

                new_points = [points[idx] for idx in current_simplex] + [points[k]]

                MEB = minimal_enclosing_sphere(new_points)

                if MEB.radius < l:
                    simplex[pn] = MEB
                    IsSimplex[pn] = 1
                else:
                    IsSimplex[pn] = 0

    for key, value in simplex.items():
        print(f"{key} -> {value.radius}")

In [9]:
def test_task3():
    P = [(5, 0, 1), (-1, -3, 4), (-1, -4, -3), (-1, 4, -3)]
    task3(P,1000)

test_task3()

(0,) -> 0
(1,) -> 0
(2,) -> 0
(3,) -> 0
(1, 0) -> 3.6742346141747673
(2, 0) -> 4.123105625617661
(2, 1) -> 3.5355339059327378
(3, 0) -> 4.123105625617661
(3, 1) -> 4.949747468305833
(3, 2) -> 4.0
(1, 2, 0) -> 4.395245364957663
(1, 3, 0) -> 5.049752469181039
(2, 3, 0) -> 4.714951667914447
(2, 3, 1) -> 5.0
(1, 2, 3, 0) -> 5.0990195135927845


In [10]:
def Is_in_alpha_complex(P,R):
    """Check if the simplex R is in the alpha complex P"""
    
    if len(R)>len(P[0]):
        return False

    #On suppose que le simplex est un simplexe
    if not R or len(P)<=len(P[0])+1  :
        return True

    MEB=make_sphere_n_points(R)
    
    for p in P:
       if MEB.contains(p):
           return False

    return True

def filtration_value(R):
    """Compute the filtration value of the simplex R"""
    return make_sphere_n_points(R).radius

def task4(points,R):
    """"Reuse the LP-type algorithm with new parameters in order to determine
if a simplex is in the α-complex and its filtration value. Note that this is less
standard than for the MEB, you need to explain how this new problem fits in
the framework."""

    """On part du principe que pour trouver le cercle le plus petit possible qui a ces points sur sa frontière,
    il suffit de calculer leur MEB (qui a nécessairement 2 points sur sa frontière) et de voir si le MEB a tous les points sur sa frontière"""
    
    return Is_in_alpha_complex(points,R),filtration_value(R)

In [11]:
def test_task4():
    P=[(0,5,0),(3,4,0),(-3,4,0)]
    R=P
    print(f"---- Test for {P}")
    a= task4(P,R)
    print(f"Complex ? {a[0]}")
    print(f"filtration value: {a[1]}")

    P.append((0,0,4))
    R=P
    print(f"---- Test for {P}")
    a= task4(P,R)
    print(f"Complex ? {a[0]}")
    print(f"filtration value: {a[1]}")

    P.append((0,0,-4))
    print(f"---- Test for {P}")
    a= task4(P,R)
    print(f"Complex ? {a[0]}")
    print(f"filtration value: {a[1]}")

test_task4()

---- Test for [(0, 5, 0), (3, 4, 0), (-3, 4, 0)]
Complex ? True
filtration value: 5.000000000000001
---- Test for [(0, 5, 0), (3, 4, 0), (-3, 4, 0), (0, 0, 4)]
Complex ? True
filtration value: 5.125000000000002
---- Test for [(0, 5, 0), (3, 4, 0), (-3, 4, 0), (0, 0, 4), (0, 0, -4)]
Complex ? False
filtration value: 3.2882366094914763


In [12]:
def task5(points,K,l):
      """ Given a set P of n points in Rd, implement an algorithm that enu-
      merates the simplexes of dimension at most k and filtration value at most l of
      the α-complex and their filtration values.""" 
      enum = enum3(points)
      print(f"enum={enum}")
      filtration_value=0
      IsSimplex = {tuple([i]): 1 for i in range(len(points))}

      if K>=len(points[0]):
         K=len(points[0])

      simplex = {tuple([i]): Sphere(points[i], 0) for i in range(len(points))} #on initialise le premier simplexe

      for i in range(1,K):
        for j in range(len(enum[i-1])):
              
          current_simplex = enum[i-1][j]

          for k in range(len(points)):
                  
            if k in current_simplex:
              break
                  
            pn =tuple(current_simplex + [k])

            if not Is_in_alpha_complex(points,pn):
              IsSimplex[pn] = 0
              break

            if pn in simplex:
              break

            new_simplex = [points[idx] for idx in current_simplex] + [points[k]]

            MEB = trivial(new_simplex)

            if MEB.radius < l:
              if MEB.radius > filtration_value:
                        filtration_value=MEB.radius
              simplex[pn] = MEB
              IsSimplex[pn] = 1
            else:
              IsSimplex[pn] = 0
              break

            print(f"new simplex: {pn} -> {MEB.radius} ")
      print(f"filtration value: {filtration_value}")


In [15]:
def test_task5():
 #generate random n points in R^d:
 n=random.randint(5,10)
 print(f"n={n}")
 d=random.randint(2,5)
 print(f"d={d}")
 points=[tuple(np.random.rand(d)) for i in range(n)]
 print(f"Points: {points}")
 k=random.randint(2,d)
 print(f"k={k}")
 l=np.random.rand(1)
 print(f"l={l}")
 task5(points,k,l)
 #task3(points,l)

test_task5()


n=8
d=2
Points: [(0.9273162911159152, 0.6120440039352517), (0.9249128074686029, 0.6821321146110241), (0.5982296271510951, 0.09731416306099683), (0.13336590321537556, 0.3952489477346166), (0.28825732826065653, 0.5454015144182559), (0.1657882612910051, 0.3834184246007034), (0.20942701040974787, 0.5094458846569597), (0.28779895275870415, 0.8329231263704363)]
k=2
l=[0.6799618]
enum=[[[0], [1], [2], [3], [4], [5], [6], [7]], [[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [3, 4], [3, 5], [3, 6], [3, 7], [4, 5], [4, 6], [4, 7], [5, 6], [5, 7], [6, 7]], [[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 1, 5], [0, 1, 6], [0, 1, 7], [0, 2, 3], [0, 2, 4], [0, 2, 5], [0, 2, 6], [0, 2, 7], [0, 3, 4], [0, 3, 5], [0, 3, 6], [0, 3, 7], [0, 4, 5], [0, 4, 6], [0, 4, 7], [0, 5, 6], [0, 5, 7], [0, 6, 7], [1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 2, 6], [1, 2, 7], [1, 3, 4], [1, 3, 5], [1, 3, 6], [1, 3, 7], [1, 4, 5], [1, 4, 6

In [ ]:
def task_visiulization():
    #a completer: creer des points radonnées et les afficher avec les simplexes de l'alpha complexe et leur filtration value et Cech complexe en differentes couleurs
    pass


A finir :
* task_visiulization
* la partie theorique
* enum est trop couteuse $O(2^n)$ 
* dans test_task5(),task3 ne marche pas
* des explications pour chaque questions